In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
import os
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig, DebertaTokenizer, BertTokenizer, TFAutoModel
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import TFDebertaModel
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold

tokenizers.__version__: 0.12.1
transformers.__version__: 4.20.1


2022-08-15 09:06:22.139965: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-15 09:06:22.141163: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-15 09:06:22.141842: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-15 09:06:22.143842: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [2]:
label_mapping = {"Adequate":0, "Effective":1, "Ineffective":2}

In [3]:
TEST_DIR = "../input/feedback-prize-effectiveness"
TRAIN_FOLDS_DIR = '../input/feedbacktrainfolds'
train = pd.read_csv(f"{TRAIN_FOLDS_DIR}/train_folds.csv")
test = pd.read_csv(f"{TEST_DIR}/test.csv")
submission = pd.read_csv(f"{TEST_DIR}/sample_submission.csv")

# Without Essay text- Bert

In [4]:
tokenizer = BertTokenizer.from_pretrained("../input/huggingface-bert-variants/bert-base-cased/bert-base-cased")

In [5]:
train['inputs'] = train.discourse_type + tokenizer.sep_token +train.discourse_text
test['inputs'] = test.discourse_type + tokenizer.sep_token +test.discourse_text
def build_model(fold, bert_model, max_len=256):  
    callbacks = tf.keras.callbacks.ModelCheckpoint(f'./model{fold}_weights.h5',
                                                   save_weights_only=True, save_best_only=True)
    input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    token_type_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")
    attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    sequence_output = bert_model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
    clf_output = sequence_output[:, 0, :]
    clf_output = tf.keras.layers.Dropout(.1)(clf_output)
    out = tf.keras.layers.Dense(3, activation='softmax')(clf_output)
    
    model = tf.keras.models.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model, callbacks

bert_model = (transformers.TFBertModel.from_pretrained('../input/huggingface-bert-variants/bert-base-cased/bert-base-cased'))

Some layers from the model checkpoint at ../input/huggingface-bert-variants/bert-base-cased/bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at ../input/huggingface-bert-variants/bert-base-cased/bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [6]:
def test_tokenizer_encode(texts, tokenizer):
    
    input_ids = np.zeros((test.shape[0], 256))
    token_type_ids = np.zeros((test.shape[0], 256))
    attention_mask_ids = np.zeros((test.shape[0], 256))
    
    for e,text in enumerate(texts):
        tokenized_text = tokenizer.encode_plus(text, add_special_tokens=True,
                              padding='max_length',max_length=256,truncation= True)
        input_ids[e,:] = tokenized_text['input_ids']
        token_type_ids[e,:] = tokenized_text['token_type_ids']
        attention_mask_ids[e,:] = tokenized_text['attention_mask']

    return input_ids, token_type_ids, attention_mask_ids

In [7]:
X_test = test_tokenizer_encode(test['inputs'].astype(str), tokenizer)

In [8]:
AUTO = tf.data.experimental.AUTOTUNE

In [9]:
prediction_list = []
for fold in range(5):
    X_test_dataset = (X_test[0][:,:], X_test[1][:,:],
              X_test[2][:,:])
    test_dataset = (
        tf.data.Dataset
        .from_tensor_slices((X_test_dataset))
        .batch(16)
        .cache()
        .prefetch(AUTO)
    )
    model, callback_option = build_model(fold, bert_model, 256)
    model.load_weights(f"../input/bertcasedfeedback/model{fold}_weights.h5")
    prediction = model.predict([X_test[0],X_test[1],X_test[2]])
    prediction_list.append(prediction)

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-08-15 09:06:46.333422: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [10]:
predictions = prediction_list[0]+prediction_list[1]+ \
              prediction_list[2]+prediction_list[3]+ \
              prediction_list[4]
predictions = (predictions / 5)
test['Adequate'] = predictions[:,0]
test['Effective'] = predictions[:,1]
test['Ineffective'] = predictions[:,2]

submission_first = test[submission.columns]

In [11]:
submission_first

,discourse_id,Ineffective,Adequate,Effective
0,a261b6e14276,0.023028,0.363148,0.613824
1,5a88900e7dc1,0.033201,0.731789,0.235011
2,9790d835736b,0.013549,0.673786,0.312665
3,75ce6d68b67b,0.023127,0.480059,0.496814
4,93578d946723,0.024849,0.796667,0.178485
5,2e214524dbe3,0.020794,0.267092,0.712114
6,84812fc2ab9f,0.011068,0.132132,0.856801
7,c668ff840720,0.016864,0.579370,0.403767
8,739a6d00f44a,0.096340,0.625865,0.277795
9,bcfae2c9a244,0.009863,0.423207,0.566929


# BERT WO ESSAY AUGMENTED

In [12]:
TEST_DIR = "../input/feedback-prize-effectiveness"
TRAIN_FOLDS_DIR = '../input/feedbacktrainfolds'
train = pd.read_csv(f"{TRAIN_FOLDS_DIR}/train_folds.csv")
test = pd.read_csv(f"{TEST_DIR}/test.csv")
submission = pd.read_csv(f"{TEST_DIR}/sample_submission.csv")

In [13]:
tokenizer = BertTokenizer.from_pretrained("../input/huggingface-bert-variants/bert-base-cased/bert-base-cased")

In [14]:
train['inputs'] = train.discourse_type + tokenizer.sep_token +train.discourse_text
test['inputs'] = test.discourse_type + tokenizer.sep_token +test.discourse_text
def build_model(fold, bert_model, max_len=256):  
    callbacks = tf.keras.callbacks.ModelCheckpoint(f'./model{fold}_weights.h5',
                                                   save_weights_only=True, save_best_only=True)
    input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    token_type_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")
    attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    sequence_output = bert_model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
    clf_output = sequence_output[:, 0, :]
    clf_output = tf.keras.layers.Dropout(.1)(clf_output)
    out = tf.keras.layers.Dense(3, activation='softmax')(clf_output)
    
    model = tf.keras.models.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model, callbacks

bert_model = (transformers.TFBertModel.from_pretrained('../input/huggingface-bert-variants/bert-base-cased/bert-base-cased'))

Some layers from the model checkpoint at ../input/huggingface-bert-variants/bert-base-cased/bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at ../input/huggingface-bert-variants/bert-base-cased/bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [15]:
def test_tokenizer_encode(texts, tokenizer):
    
    input_ids = np.zeros((test.shape[0], 256))
    token_type_ids = np.zeros((test.shape[0], 256))
    attention_mask_ids = np.zeros((test.shape[0], 256))
    
    for e,text in enumerate(texts):
        tokenized_text = tokenizer.encode_plus(text, add_special_tokens=True,
                              padding='max_length',max_length=256,truncation= True)
        input_ids[e,:] = tokenized_text['input_ids']
        token_type_ids[e,:] = tokenized_text['token_type_ids']
        attention_mask_ids[e,:] = tokenized_text['attention_mask']

    return input_ids, token_type_ids, attention_mask_ids

In [16]:
X_test = test_tokenizer_encode(test['inputs'].astype(str), tokenizer)

In [17]:
AUTO = tf.data.experimental.AUTOTUNE

In [18]:
prediction_list = []
for fold in range(5):
    X_test_dataset = (X_test[0][:,:], X_test[1][:,:],
              X_test[2][:,:])
    test_dataset = (
        tf.data.Dataset
        .from_tensor_slices((X_test_dataset))
        .batch(16)
        .cache()
        .prefetch(AUTO)
    )
    model, callback_option = build_model(fold, bert_model, 256)
    model.load_weights(f"../input/augmented-bert-woessay/model{fold}_weights.h5")
    prediction = model.predict([X_test[0],X_test[1],X_test[2]])
    prediction_list.append(prediction)

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [19]:
predictions = prediction_list[0]+prediction_list[1]+ \
              prediction_list[2]+prediction_list[3]+ \
              prediction_list[4]
predictions = (predictions / 5)
test['Adequate'] = predictions[:,0]
test['Effective'] = predictions[:,1]
test['Ineffective'] = predictions[:,2]

submission_third = test[submission.columns]

In [20]:
submission_third

,discourse_id,Ineffective,Adequate,Effective
0,a261b6e14276,0.027848,0.483920,0.488232
1,5a88900e7dc1,0.053030,0.635413,0.311557
2,9790d835736b,0.036607,0.541733,0.421660
3,75ce6d68b67b,0.073832,0.389516,0.536652
4,93578d946723,0.029687,0.505888,0.464424
5,2e214524dbe3,0.018476,0.332572,0.648951
6,84812fc2ab9f,0.017008,0.281306,0.701685
7,c668ff840720,0.022768,0.414298,0.562935
8,739a6d00f44a,0.050354,0.495189,0.454457
9,bcfae2c9a244,0.013215,0.486243,0.500543


# ROBERTA ESSAY

In [21]:
TEST_DIR = "../input/feedback-prize-effectiveness"
TRAIN_FOLDS_DIR = '../input/feedbacktrainfolds'
train = pd.read_csv(f"{TRAIN_FOLDS_DIR}/train_folds.csv")
test = pd.read_csv(f"{TEST_DIR}/test.csv")
submission = pd.read_csv(f"{TEST_DIR}/sample_submission.csv")

In [22]:
def get_essay_text(essay_id, directory):
    parent_path = TEST_DIR + "/" + directory
    essay_path = os.path.join(parent_path, f"{essay_id}.txt")
    essay_text = open(essay_path, 'r').read()
    return essay_text
train['essay_text'] = train['essay_id'].apply(get_essay_text, directory='train')
test['essay_text'] = test['essay_id'].apply(get_essay_text, directory='test')

In [23]:
from text_unidecode import unidecode
from typing import Dict, List, Tuple
import codecs
def replace_encoding_with_utf8(error: UnicodeError) -> Tuple[bytes, int]:
    return error.object[error.start : error.end].encode("utf-8"), error.end


def replace_decoding_with_cp1252(error: UnicodeError) -> Tuple[str, int]:
    return error.object[error.start : error.end].decode("cp1252"), error.end

# Register the encoding and decoding error handlers for `utf-8` and `cp1252`.
codecs.register_error("replace_encoding_with_utf8", replace_encoding_with_utf8)
codecs.register_error("replace_decoding_with_cp1252", replace_decoding_with_cp1252)

def resolve_encodings_and_normalize(text: str) -> str:
    """Resolve the encoding problems and normalize the abnormal characters."""
    text = (
        text.encode("raw_unicode_escape")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
        .encode("cp1252", errors="replace_encoding_with_utf8")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
    )
    text = unidecode(text)
    return text

train['discourse_text'] = train['discourse_text'].apply(resolve_encodings_and_normalize)
train['essay_text'] = train['essay_text'].apply(resolve_encodings_and_normalize)

test['discourse_text'] = test['discourse_text'].apply(resolve_encodings_and_normalize)
test['essay_text'] = test['essay_text'].apply(resolve_encodings_and_normalize)

In [24]:
train['text'] = train['discourse_type'] + " [SEP] " + train['discourse_text'] + " [SEP] " + train['essay_text']
test['text'] = test['discourse_type'] + " [SEP] " + test['discourse_text'] + " [SEP] " + test['essay_text']

In [25]:
tokenizer = AutoTokenizer.from_pretrained("../input/robertalarge")

In [26]:
def create_model():
    input_id = tf.keras.layers.Input(shape = (512) , dtype = tf.int32, name = 'input_ids')
    attention_mask = tf.keras.layers.Input(shape = (512), dtype = tf.int32, name = 'attention_mask')
    
    transformer_model = transformers.TFAutoModel.from_pretrained('../input/robertalarge', from_pt=True)
    cls_token = transformer_model(input_ids = input_id , attention_mask = attention_mask)[0][:,0,:]
    
    prediction = tf.keras.layers.Dense(3 , activation = "softmax")(cls_token)

    return tf.keras.models.Model(inputs = [input_id, attention_mask] , outputs = prediction)
import tensorflow.keras.backend as K
df_test = test.copy()

In [27]:
@tf.function
def test_map_function(encodings):
    input_ids = encodings['input_ids']
    attention_mask = encodings['attention_mask']
    
    
    return {'input_ids': input_ids , 'attention_mask': attention_mask}

In [28]:
strategy = tf.distribute.get_strategy()

In [29]:
histories = []
scores = []
preds_list = []
for fold in range(4):
    print(f"====== FOLD RUNNING {fold}======")
    
    X_test = df_test['text']
    
    print(" Test Generating Tokens")
    test_embeddings = tokenizer(
        X_test.tolist(),
        truncation = True, 
        padding = 'max_length',
        max_length =512  
    )
    
    print("Test Generating Dataset")
    test = tf.data.Dataset.from_tensor_slices((test_embeddings))
    test = (
                test
                .map(test_map_function, num_parallel_calls= AUTO)
                .batch(24)
                .prefetch(AUTO)
            )
    
    #Clearing backend session
    K.clear_session()
    print("Backend Cleared")

    print("Model Creation")
    with strategy.scope():
        model = create_model()
        model.load_weights(f'../input/roberta-essay-feedback/roberta{fold}_weights.h5')

    print("prediction on test data")
    preds = model.predict(test , verbose = 1)
    preds_list.append(preds)
    
    del model, X_test, test , test_embeddings

====== FOLD RUNNING 0======
 Test Generating Tokens
Test Generating Dataset
Backend Cleared
Model Creation


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


prediction on test data
1/1 [==============================] - 6s 6s/step
====== FOLD RUNNING 1======
 Test Generating Tokens
Test Generating Dataset
Backend Cleared
Model Creation


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


prediction on test data
1/1 [==============================] - 6s 6s/step
====== FOLD RUNNING 2======
 Test Generating Tokens
Test Generating Dataset
Backend Cleared
Model Creation


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


prediction on test data
1/1 [==============================] - 5s 5s/step
====== FOLD RUNNING 3======
 Test Generating Tokens
Test Generating Dataset
Backend Cleared
Model Creation


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


prediction on test data
1/1 [==============================] - 6s 6s/step


In [30]:
predictions = preds_list[0]+preds_list[1]+ \
              preds_list[2]+preds_list[3]
predictions = (predictions / 4)
df_test['Adequate'] = predictions[:,0]
df_test['Effective'] = predictions[:,1]
df_test['Ineffective'] = predictions[:,2]

In [31]:
submission_second = df_test[submission.columns]

# Final Submission combined

In [32]:
submission_first.columns

Index(['discourse_id', 'Ineffective', 'Adequate', 'Effective'], dtype='object')

In [33]:
submission_final = pd.merge(submission_first, submission_second[['discourse_id','Ineffective','Adequate','Effective']],
                            on='discourse_id',how='inner')
submission_final = pd.merge(submission_final, submission_third[['discourse_id','Ineffective','Adequate','Effective']],
                            on='discourse_id',how='inner')

In [34]:
submission_final.head()

,discourse_id,Ineffective_x,Adequate_x,Effective_x,Ineffective_y,Adequate_y,Effective_y,Ineffective,Adequate,Effective
0,a261b6e14276,0.023028,0.363148,0.613824,0.038588,0.426542,0.534870,0.027848,0.483920,0.488232
1,5a88900e7dc1,0.033201,0.731789,0.235011,0.026656,0.529459,0.443885,0.053030,0.635413,0.311557
2,9790d835736b,0.013549,0.673786,0.312665,0.035395,0.433014,0.531590,0.036607,0.541733,0.421660
3,75ce6d68b67b,0.023127,0.480059,0.496814,0.036914,0.440101,0.522985,0.073832,0.389516,0.536652
4,93578d946723,0.024849,0.796667,0.178485,0.038124,0.443226,0.518650,0.029687,0.505888,0.464424


In [35]:
submission_final.columns = ['discourse_id','Ineffective_x',
                            'Adequate_x','Effective_x','Ineffective_y',
                            'Adequate_y','Effective_y','Ineffective_z',
                            'Adequate_z','Effective_z']

In [36]:
submission_final['Adequate'] = (submission_final['Adequate_x'] + 
                                submission_final['Adequate_y'] +
                                submission_final['Adequate_z'])/3
submission_final['Ineffective'] = (submission_final['Ineffective_x'] + 
                                   submission_final['Ineffective_y'] +
                                   submission_final['Ineffective_z'])/3
submission_final['Effective'] = (submission_final['Effective_x'] + 
                                 submission_final['Effective_y'] + 
                                 submission_final['Effective_z'])/3

In [37]:
submission_final

,discourse_id,Ineffective_x,Adequate_x,Effective_x,Ineffective_y,Adequate_y,Effective_y,Ineffective_z,Adequate_z,Effective_z,Adequate,Ineffective,Effective
0,a261b6e14276,0.023028,0.363148,0.613824,0.038588,0.426542,0.534870,0.027848,0.483920,0.488232,0.424537,0.029821,0.545642
1,5a88900e7dc1,0.033201,0.731789,0.235011,0.026656,0.529459,0.443885,0.053030,0.635413,0.311557,0.632220,0.037629,0.330151
2,9790d835736b,0.013549,0.673786,0.312665,0.035395,0.433014,0.531590,0.036607,0.541733,0.421660,0.549511,0.028517,0.421972
3,75ce6d68b67b,0.023127,0.480059,0.496814,0.036914,0.440101,0.522985,0.073832,0.389516,0.536652,0.436559,0.044624,0.518817
4,93578d946723,0.024849,0.796667,0.178485,0.038124,0.443226,0.518650,0.029687,0.505888,0.464424,0.581927,0.030887,0.387186
5,2e214524dbe3,0.020794,0.267092,0.712114,0.029021,0.422849,0.548129,0.018476,0.332572,0.648951,0.340838,0.022764,0.636398
6,84812fc2ab9f,0.011068,0.132132,0.856801,0.031856,0.349725,0.618419,0.017008,0.281306,0.701685,0.254388,0.019977,0.725635
7,c668ff840720,0.016864,0.579370,0.403767,0.039587,0.412665,0.547749,0.022768,0.414298,0.562935,0.468777,0.026406,0.504817
8,739a6d00f44a,0.096340,0.625865,0.277795,0.036871,0.338430,0.624699,0.050354,0.495189,0.454457,0.486495,0.061188,0.452317
9,bcfae2c9a244,0.009863,0.423207,0.566929,0.032724,0.478928,0.488348,0.013215,0.486243,0.500543,0.462793,0.018601,0.518607


In [38]:
submission_final[submission.columns].to_csv("submission.csv", index=False)